# Dev le(s) modèle(s) de désambiguïsation lexicale

In [1]:
import re
import math
import nltk
import random
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition
from xml.dom.minidom import parse
from gensim.models import Word2Vec
from nltk.cluster.kmeans import KMeansClusterer

trial_corpus_path = "trial_corpus.xml"
test_corpus_path = "test_corpus.xml"

In [2]:
def loadCorpus(path):
    """Load a formatted corpus data file

    Parameters
    ----------
    path: str
        Path to the corpus xml file to load


    Returns
    -------
    list
        A 3 dimensional list containing for each document, the sentences that it is composed of.
        Where each sentence is a list of single tokens.
    dict
        A dictionnary mapping a lemma with it's document/sentence/index position and the BabelNet Sense attributed
    """

    DOMTree = parse(path)

    documents = []
    sens_dict = {}
    for doc in DOMTree.getElementsByTagName("document"):
        # For each document
        sentences = []
        for sent in doc.getElementsByTagName("sentence"):
            # And for each sentence
            # Append the new sentence
            s = sent.getAttribute("s")
            sentences.append(s.split())

            # Map the lemmas in the sentence with it's doc/sentence/index position and BabelNet sense
            for lem in sent.getElementsByTagName("lemma"):
                idx = lem.getAttribute("idx")
                lemma = lem.getAttribute("lemma")
                # Few lemma may have more than 1 BabelNet sense (due to redundancy in BN)
                # Only keep the 1st one
                sense = lem.getAttribute("senses").split()[0] 
                
                ctx = (int(doc.getAttribute("id")),
                        int(sent.getAttribute("id")),
                        int(idx),
                        sense)
                if not lemma in sens_dict:
                    sens_dict[lemma] = [ctx]
                else:
                    sens_dict[lemma].append(ctx)

        documents.append(sentences)

    return (documents, sens_dict)


documents, sens_dict = loadCorpus(trial_corpus_path)


print("Documents (%d):"%(len(documents)))
for i, d in enumerate(documents):
    print("\tDoc %2d: %02d sentences"%(i, len(d)))

print("\nLemmas (%d):"%(len(sens_dict)))
for i, (k, v) in zip(range(10), sens_dict.items()):
    print("\tLemma %d: %s -> %d values"%(i, k, len(v)))
print("\t...\n")
print("\nEx. lemma 0: ")
print("\t", list(sens_dict.keys())[0], "->", list(sens_dict.values())[0])

Documents (1):
	Doc  0: 36 sentences

Lemmas (124):
	Lemma 0: guerre_contre_la_drogue -> 1 values
	Lemma 1: Amérique_Latine -> 2 values
	Lemma 2: presse -> 9 values
	Lemma 3: mois -> 2 values
	Lemma 4: journaliste -> 5 values
	Lemma 5: trafiquant_de_drogue -> 2 values
	Lemma 6: guérillero -> 1 values
	Lemma 7: gauche -> 2 values
	Lemma 8: personne -> 1 values
	Lemma 9: Colombie -> 4 values
	...


Ex. lemma 0: 
	 guerre_contre_la_drogue -> [(0, 0, 7, 'bn:00028885n')]


# Word2vec

In [3]:
def documentsSentences(doc):
    docSentences = []
    for d in doc:
        for s in d:
            docSentences.append(s)
    return docSentences

def createW2vModel():
    return Word2Vec(documentsSentences(documents), min_count=1)

try:
    w2v = Word2Vec.load("w2v.model")
except FileNotFoundError:
    w2v = createW2vModel()
    w2v.save("w2v.model")

print("w2v Vocab size:", len(w2v.wv.vocab))

w2v Vocab size: 429


# Huang

In [4]:
def lemma2Senses(lemma):
    senses = [bn for _,_,_,bn in sens_dict[lemma]]
    return list(set(senses))

print("\nEx.", list(sens_dict.keys())[0])
print("\t->", list(sens_dict.values())[0])
print("\t->", lemma2Senses(list(sens_dict.keys())[0]))


Ex. guerre_contre_la_drogue
	-> [(0, 0, 7, 'bn:00028885n')]
	-> ['bn:00028885n']


## Visualisation rapide des lemmes à désambiguïser
On remarque que peu de lemmes sont associés à plusieurs sens. Certains apparaissent plusieurs fois avec toujours le meme sens. Pire ! D'autres n'apparaissent qu'une seule fois.<br/>
Il est aussi intéressant de remarquer que certains lemmes sont associés 9 fois avec le sens_1 et 1 fois avec le sens_2. Ceci peut trouver son origine dans les annotations via BabelNet qui propose différents sens redondants d'un mot.<br/>
Pour exemple, le lemme <i>journaliste</i> est associé aux sens BabelNet suivants :
<ol>
    <li>bn:00048461n : celui qui recueille, écrit ou distribue des informations</li>
    <li>bn:00057562n : celui qui enquête, rapporte ou rédige les actualités</li>
</ol>

In [5]:
polysem = {}
solo = {}
npolysem = {}

for k,v in sens_dict.items():
    if len(v) == 1:
        solo[k] = v
        continue
    
    _,_,_,sense_bn = v[0]
    poly = False
    for _,_,_,bn in v:
        if bn != sense_bn:
            polysem[k] = v
            poly = True
            break
    if not poly:
        npolysem[k] = v

print("Nb lemmas:", len(sens_dict))
print()

print("polysems:", len(polysem))
for _, (k, v) in zip(range(5), polysem.items()):
    print("\t", k, " -> ", v)

print("\nsolo:", len(solo))
for _, (k, v) in zip(range(5), solo.items()):
    print("\t", k, " -> ", v)

print("\nnon polysem:", len(npolysem))
for _, (k, v) in zip(range(5), npolysem.items()):
    print("\t", k, " -> ", v)

Nb lemmas: 124

polysems: 3
	 journaliste  ->  [(0, 1, 8, 'bn:00048461n'), (0, 2, 10, 'bn:00048461n'), (0, 16, 43, 'bn:00048461n'), (0, 18, 38, 'bn:00057562n'), (0, 19, 27, 'bn:00048461n')]
	 contrôle  ->  [(0, 3, 29, 'bn:00022287n'), (0, 26, 19, 'bn:00022283n')]
	 journal  ->  [(0, 5, 30, 'bn:00057563n'), (0, 6, 5, 'bn:00057563n'), (0, 7, 11, 'bn:00057563n'), (0, 8, 19, 'bn:00057564n'), (0, 18, 5, 'bn:00057563n')]

solo: 86
	 guerre_contre_la_drogue  ->  [(0, 0, 7, 'bn:00028885n')]
	 guérillero  ->  [(0, 1, 22, 'bn:02557244n')]
	 personne  ->  [(0, 1, 33, 'bn:00046516n')]
	 année  ->  [(0, 2, 5, 'bn:00078738n')]
	 août  ->  [(0, 3, 18, 'bn:00007140n')]

non polysem: 35
	 Amérique_Latine  ->  [(0, 0, 9, 'bn:00050165n'), (0, 4, 14, 'bn:00050165n')]
	 presse  ->  [(0, 0, 23, 'bn:00064245n'), (0, 4, 38, 'bn:00064245n'), (0, 12, 4, 'bn:00064245n'), (0, 16, 29, 'bn:00064245n'), (0, 17, 19, 'bn:00064245n'), (0, 19, 19, 'bn:00064245n'), (0, 20, 9, 'bn:00064245n'), (0, 21, 28, 'bn:00064245n'),

## Implémentation de la méthode proposée par Huang
<ol>
    <li>Collecte les fenetres d'occurrence d'un mot </li>
    <li>Calcule le vecteur de contexte, moyenne des vecteurs-mots de chaque mots dans un contexte</li>
    <li>Cluster les vecteurs de contextes (spherical K-means)</li>
    <li>Associe à chaque cluster un sens</li>
</ol>

In [6]:
def meanSentenceVector(w2vModel, sentence):
    return sumSentenceVector(w2vModel, sentence)
    #return np.array([w2vModel.wv[word] for word in sentence]).mean(axis=0)

def sumSentenceVector(w2vModel, sentence):
    return np.array([w2vModel.wv[word] for word in sentence]).sum(axis=0)

In [7]:
ctx_w = 11 # Contexte window size

global_truth = []
global_classif = []

for lemma, senses in polysem.items():
    labels = lemma2Senses(lemma)
    num_senses = len(labels)

    # Map clusters with a value from, 0 to num_senses-1
    truth = [labels.index(bn) for _,_,_,bn in senses]
    global_truth.append(truth)

    mean_vectors = []
    for d,s,i,_ in senses:
        l = len(documents[d][s])
        # Extract the words in the contexte window
        window = documents[d][s][max(0, i-math.floor((ctx_w-1)/2)) : min(l, i+math.ceil((ctx_w-1)/2))+1]

        # Compute the context vector (mean of the words vectors in the window)
        mean_vectors.append(meanSentenceVector(w2v, window))

    # Spherical K-means clustering
    skm = KMeansClusterer(num_senses, nltk.cluster.util.cosine_distance, rng=random.Random(0), repeats=10)
    assigned_clusters = skm.cluster(mean_vectors, assign_clusters=True)
    global_classif.append(assigned_clusters)

    print(truth)
    print(assigned_clusters)
    print()

#print(sklearn.metrics.classification_report([y for x in global_truth for y in x], [y for x in global_classif for y in x]))


[0, 0, 0, 1, 0]
[0, 1, 0, 1, 1]

[1, 0]
[0, 1]

[1, 1, 1, 0, 1]
[0, 0, 0, 1, 1]



### Associe à chaque cluster un sens

In [8]:
def argsmax(lst):
    max = lst[0]
    argsmax = []
    for i in range(len(lst)):
        if lst[i] == max:
            argsmax.append(i)
        elif lst[i] > max:
            max = lst[i]
            argsmax = [i]
    return argsmax

def couple(arr1, arr2):
    # truth, classif
    l = len(arr1)
    nb_c = len(set(arr1))
    arr = np.zeros((nb_c,nb_c), dtype=int)

    for i in range(l):
        arr[arr2[i],arr1[i]] += 1

    cs = [i for i in range(len(arr))]
    ts = [i for i in range(len(arr))]
    map = {}

    for iter in range(len(arr)):
        temp = len(arr)
        for i in range(len(arr)):
            c = arr[i]
            ams = argsmax(c)
            if len(ams) == 1:
                ams = ams[0]
                map[cs[i]] = ts[ams]
                arr = np.delete(np.delete(arr, ams, 1), i, 0)
                cs = np.delete(cs, i)
                ts = np.delete(ts, ams)
                break
        
        if len(arr) == temp:
            j = np.argmax(arr[0])
            map[cs[0]] = ts[j]
            arr = np.delete(np.delete(arr, j, 1), 0, 0)
            cs = np.delete(cs, 0)
            ts = np.delete(ts, j)

    return map

couple(global_truth[0], global_classif[0])

{0: 0, 1: 1}

## Premiers Résultats

In [9]:
final_classif = []
for i in range(len(global_classif)):
    cluster_tags = couple(global_truth[i], global_classif[i])
    final_classif.append([cluster_tags[i] for i in global_classif[i]])

    print(list(polysem.keys())[i])
    print(list(polysem.values())[i])
    print(final_classif[i])
    print(global_truth[i])
    print()

journaliste
[(0, 1, 8, 'bn:00048461n'), (0, 2, 10, 'bn:00048461n'), (0, 16, 43, 'bn:00048461n'), (0, 18, 38, 'bn:00057562n'), (0, 19, 27, 'bn:00048461n')]
[0, 1, 0, 1, 1]
[0, 0, 0, 1, 0]

contrôle
[(0, 3, 29, 'bn:00022287n'), (0, 26, 19, 'bn:00022283n')]
[1, 0]
[1, 0]

journal
[(0, 5, 30, 'bn:00057563n'), (0, 6, 5, 'bn:00057563n'), (0, 7, 11, 'bn:00057563n'), (0, 8, 19, 'bn:00057564n'), (0, 18, 5, 'bn:00057563n')]
[1, 1, 1, 0, 0]
[1, 1, 1, 0, 1]



Score associations clustering/gold truth

In [10]:
print(sklearn.metrics.classification_report([y for x in global_truth for y in x], [y for x in final_classif for y in x]))

              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.71      0.83      0.77         6

    accuracy                           0.75        12
   macro avg       0.76      0.75      0.75        12
weighted avg       0.76      0.75      0.75        12



In [11]:
import csv

def load_dictionary(path):
    dictionary_dict = {}

    with open(path, "r") as file:
        file.readline()
        for row in file:
            row = row.split(";")
            lemma = row.pop(0)
            nb = int(row.pop(0))
            
            # Make sure the definition is not empty...
            if nb > 0:
                ids = (row.pop(0)).split(",")
                defs = (";".join(row)).split("\",\"")
                temp = defs[0].split(",\"")
                defs = temp + defs[1:]

                dictionary_dict[lemma] = (ids, defs)
    
    return dictionary_dict

dictionary = load_dictionary("dict.dictionary")
print("Dictionary length:", len(dictionary))

Dictionary length: 119


In [12]:
dictionary["mardi"]

(['bn:00078546n', 'bn:00118342n'],
 ['"Le mardi est le jour de la semaine qui succède au lundi et qui précède le mercredi. Jour de la semaine Le deuxième jour de la semaine en Europe et dans les pays utilisant la norme ISO 8601; le troisième jour de la semaine aux États-Unis d\'Amérique.',
  'Mardi est le troisième livre publié par l\'écrivain américain Herman Melville."\n'])

Apprend les embeddings avec les definitions des mots

Word2Vec

In [13]:
def definitionsSentences():
    defSentences = []
    for _,defs in dictionary.values():
        for s in defs:
            defSentences.append(s.split())
    return defSentences

def createExtendedW2vModel():
    return Word2Vec(documentsSentences(documents)+definitionsSentences(), min_count=1)

try:
    w2v = Word2Vec.load("w2v.model.extended")
except FileNotFoundError:
    w2v = createExtendedW2vModel()
    w2v.save("w2v.model.extended")

print("Extended w2v Vocab size:", len(w2v.wv.vocab))

Extended w2v Vocab size: 6628


Calcule des vecteurs de définitions

In [14]:
defsVectors = {} #map id BN -> vecteur

for k,(ids,defs) in dictionary.items():
    defsVectors[k] = np.array([meanSentenceVector(w2v,d.split()) for d in defs]).reshape((len(defs), 100))

In [15]:
print(sens_dict["état"])
print(dictionary["état"])

[(0, 16, 23, 'bn:00074009n')]
(['bn:00031647n', 'bn:00063703n', 'bn:03719156n', 'bn:00023235n', 'bn:00964536n', 'bn:00007255n', 'bn:01606252n', 'bn:03347921n', 'bn:00074011n', 'bn:02220665n', 'bn:00026336n', 'bn:02108701n', 'bn:00889834n', 'bn:00031648n', 'bn:00023236n', 'bn:03157857n', 'bn:04721822n', 'bn:00064914n', 'bn:00029464n', 'bn:13858447n', 'bn:01110993n'], ['"Le patrimoine ou patrimoine net d\'un individu, d\'une famille ou d\'un organisme est : le patrimoine brut, ensemble des biens qu\'il possède, évalués au prix du marché ; moins ses dettes et autres engagements financiers. Ensemble des biens d\'\'une personne ou entité', "Le droit personnel, ou droit de créance est le droit d'exiger d'une personne, le débiteur, une obligation quelconque, que détient le créancier.", "En gravure, un état est une forme différente d'une estampe, causée par un changement délibéré et permanent effectué sur une matrice telle qu'une plaque de cuivre ou un bloc de bois.", "Un Land est un État fédé

In [16]:
csim = w2v.wv.cosine_similarities(w2v.wv["état"], defsVectors["état"])
print(csim)
print(np.argmax(csim))
print(dictionary["état"][1][np.argmax(csim)])

[0.99110925 0.9911546  0.9912008  0.9911021  0.9910225  0.99105155
 0.99114907 0.9910726  0.99116457 0.991084   0.99093914 0.99117416
 0.991151   0.991062   0.9910879  0.99111307 0.9908306  0.9910145
 0.991006   0.9911998  0.9910586 ]
2
En gravure, un état est une forme différente d'une estampe, causée par un changement délibéré et permanent effectué sur une matrice telle qu'une plaque de cuivre ou un bloc de bois.


Test association mot->sens avec les vecteurs de definitions et vecteurs de contextes en utilisant la similarité cosine

In [33]:
for i in range(len(mean_vectors)):
    csim = w2v.wv.cosine_similarities(mean_vectors[i], defsVectors["journal"])
    amax = np.argmax(csim)
    d,s,p,bn = polysem["journal"][i]
    print("Sentence:\t", " ".join(documents[d][s]))
    print("-> In doc %d s %d and pos %d (BabelNet sense: %s)"%(d, s, p, bn))
    print(" Output:")
    print("\t", dictionary["journal"][0][amax])
    print("\t", dictionary["journal"][1][amax])
    print(csim[amax])
    print("\n")

Sentence:	 Le mardi , les participants à la conférence ont été informés d' une autre atrocité , l' assassinat à Medellin de deux employés d' El_Espectador , le deuxième plus grand journal de Colombie .
-> In doc 0 s 5 and pos 30 (BabelNet sense: bn:00057563n)
 Output:
	 bn:17765228n
	 Un journal est une publication périodique recensant un certain nombre d'événements présentés sous la forme d'articles relatifs à une période donnée, généralement une journée, d'où son nom. Type de journal publié tous les jours, éventuellement six ou cinq fois par semaine
0.16082405


Sentence:	 L ’ administrateur local du journal , Luz Maria Lopez , a été abattue et sa mère blessée , tandis que sa voiture était arrêtée à un feu_rouge .
-> In doc 0 s 6 and pos 5 (BabelNet sense: bn:00057563n)
 Output:
	 bn:00048455n
	 Un palier lisse assure le guidage en rotation par glissement. Type de roulement
0.12836336


Sentence:	 Une heure plus tard , le directeur de la diffusion du journal , Miguel Soler , a été ab

In [38]:
w2v.wv.most_similar("journaux")

[('lorsque', 0.9098055958747864),
 ('nombreuses', 0.9096593856811523),
 ('Caraïbes', 0.9093672633171082),
 ('Nom', 0.9091154336929321),
 ('naturel', 0.9080072641372681),
 ('britannique', 0.9071807861328125),
 ('liberté', 0.9070574045181274),
 ('petit', 0.9070220589637756),
 ('se', 0.9063730835914612),
 ('mot', 0.9063611030578613)]